In [1]:
import os
os.chdir('/home/xyz/data/haoyan/bandits')

import bandits as bd
import numpy as np
import pandas as pd

/home/xyz/miniconda3/envs/bandit/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [51]:
from ast import literal_eval as make_tuple


l = []
names = ['random_state', 'num_exp', 'reward', 'time']
with open('/home/xyz/data/haoyan/bandits/logs/grid_search_results1.txt', 'r') as f:
    x = f.readline()
    while x:
        idx = 0
        tmp = make_tuple(x)
        tmp_dict = {}
        for k in tmp:
            if isinstance(k, dict):
                for key, val in k.items():
                    tmp_dict[key] = val
            else:
                tmp_dict[names[idx]] = k
                idx += 1
        l.append(tmp_dict)
        x = f.readline()

In [52]:
df = pd.DataFrame(l)
idx = df.groupby('random_state')['reward'].idxmax()
df.loc[idx]

,random_state,num_exp,colsample_bytree,learning_rate,max_bin,max_depth,n_estimators,reg_alpha,reg_lambda,reward,time
421,0,421,0.863658,0.094806,512,5,1000,2.936390,2.043991,-0.576138,1.064897
1052,128,452,0.915521,0.114819,1024,6,1000,2.040760,0.793623,-0.580420,1.421294
1728,256,528,0.917927,0.056005,512,6,2000,1.155490,2.046223,-0.575369,2.556232
2024,384,224,1.064792,0.066278,1024,5,2000,2.028420,2.721044,-0.567300,2.330111
2447,512,47,1.072296,0.232545,1024,6,5000,0.266377,2.245121,-0.576186,7.664883
3074,640,74,1.072431,0.081335,1024,5,2000,1.309336,2.583320,-0.583325,2.406467
3908,768,308,1.068060,0.057978,512,6,1000,1.373337,2.360026,-0.563329,1.380798
4358,896,158,1.175254,0.018641,1024,5,5000,1.308733,2.970956,-0.570461,5.720730
5215,1024,415,0.935655,0.064095,512,6,2000,0.990407,2.511235,-0.575908,2.540447
5829,1152,429,1.115057,0.056459,512,6,5000,1.719425,2.036546,-0.580197,6.299084


In [53]:
df.groupby('random_state')['reward'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
random_state,,,,,,,,,,,
0,600.0,-0.997781,0.593288,-4.153068,-1.086879,-0.738170,-0.675174,-0.633802,-0.620876,-0.593566,-0.576138
128,600.0,-0.989122,0.596928,-4.478989,-1.083013,-0.737330,-0.672203,-0.630110,-0.610437,-0.591101,-0.580420
256,600.0,-1.000343,0.623387,-4.270382,-1.043504,-0.737299,-0.671278,-0.628354,-0.612641,-0.593301,-0.575369
384,600.0,-0.981813,0.603711,-4.472518,-1.029354,-0.728119,-0.664932,-0.624645,-0.608076,-0.586853,-0.567300
512,600.0,-0.977591,0.577091,-4.144563,-1.071568,-0.744160,-0.675841,-0.636930,-0.621976,-0.593542,-0.576186
640,600.0,-0.943675,0.517570,-4.206636,-1.010626,-0.745510,-0.671490,-0.631077,-0.621697,-0.592791,-0.583325
768,600.0,-0.985830,0.584783,-3.862003,-1.034822,-0.737400,-0.670023,-0.631662,-0.614527,-0.589455,-0.563329
896,600.0,-0.998623,0.606802,-4.390122,-1.049229,-0.748293,-0.676660,-0.638701,-0.618950,-0.589258,-0.570461
1024,600.0,-1.007795,0.590069,-3.795893,-1.104404,-0.746951,-0.673566,-0.636611,-0.617810,-0.591710,-0.575908


In [55]:
import plotly.graph_objs as go

fig = go.Figure()
moving_avg_list = []
moving_lower_quantile_list = []
moving_upper_quantile_list = []
for key, gp in df.groupby('random_state'):
    moving_avg_list.append(gp.sort_values('num_exp')['reward'].rolling(100).mean().dropna().values)
    moving_lower_quantile_list.append(gp.sort_values('num_exp')['reward'].rolling(100).quantile(0.25).dropna().values)
    moving_upper_quantile_list.append(gp.sort_values('num_exp')['reward'].rolling(100).quantile(0.75).dropna().values)
moving_avg = pd.DataFrame(moving_avg_list).mean()
moving_lower_quantile = pd.DataFrame(moving_lower_quantile_list).mean()
moving_upper_quantile = pd.DataFrame(moving_upper_quantile_list).mean()
fig.add_trace(go.Scatter(x=list(range(len(moving_avg))), y=moving_avg, mode='lines', name=key))
fig.add_trace(go.Scatter(x=list(range(len(moving_lower_quantile))), y=moving_lower_quantile, mode='lines', name=key))
fig.add_trace(go.Scatter(x=list(range(len(moving_upper_quantile))), y=moving_upper_quantile, mode='lines', name=key))
fig.show()

In [5]:
from ast import literal_eval as make_tuple


l = []
last_random_state = -99999
names = ['random_state', 'num_exp', 'reward']
with open('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt', 'r') as f:
    x = f.readline()
    while x:
        tmp = make_tuple(x)
        if last_random_state != tmp[0]:
            idx = 0
            last_random_state = tmp[0]
        for element in tmp[3]:
            tmp_dict = {
                'random_state': tmp[0],
                'nth_trial': tmp[1],
                'num_exp': idx,
                'max_bin': tmp[2][0][0],
                'max_depth': tmp[2][0][1],
                'n_estimator': tmp[2][0][2],
                'time': tmp[5] / len(tmp[3]),
            }
            tmp_dict.update(element['params'])
            tmp_dict['bayesian_opt'] = element['target']
            tmp_dict['reward'] = tmp[4]
            l.append(tmp_dict)
            idx += 1
        x = f.readline()

In [15]:
df.groupby(['random_state', 'max_depth', 'max_bin', 'n_estimator'])['bayesian_opt'].agg(['mean', 'count']).sort_values('count').tail(30)

mean  count
random_state max_depth max_bin n_estimator                 
640          4         512     1000        -0.750472     50
384          7         256     2500        -0.912731     50
640          7         512     1000        -0.781259     50
256          8         256     5000        -0.669142     50
             5         512     5000        -0.722045     52
128          4         64      5000        -1.016164     52
256          5         128     500         -0.817163     52
640          5         128     500         -1.200567     52
512          8         128     500         -1.129943     52
640          6         512     500         -1.197551     52
1152         5         256     500         -1.223296     54
768          7         64      500         -0.967232     54
             5         512     5000        -0.905316     54
0            9         512     500         -1.332833     54
256          5         256     1000        -1.024112     54
768          6         512     5000        -0.694163     56
1024         7         512     2500        -0.700737     56
640          4         512     2500        -0.816159     56
128          7         512     1000        -0.900380     56
384          7         512     500         -0.881527     56
512          6         512     5000        -0.680339     56
0            7         128     5000        -0.708229     56
1152         6         512     5000        -0.880645     56
128          6         512     1000        -0.922210     56
896          8         256     2500        -0.965244     58
384          6         256     2500        -0.998466     58
768          7         256     500         -0.954472     60
1152         7         256     5000        -0.703299     64
640          7         512     2500        -0.931759     70
896          6         512     1000        -0.797851     76

In [3]:
df = pd.DataFrame(l)
df.groupby('random_state')['bayesian_opt'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
random_state,,,,,,,,,,,
0,3000.0,-0.937196,0.737505,-4.578281,-0.839434,-0.691646,-0.650089,-0.624167,-0.608778,-0.591694,-0.574700
128,3000.0,-0.901231,0.675316,-4.578281,-0.826371,-0.683980,-0.643102,-0.619138,-0.608125,-0.590300,-0.573507
256,3000.0,-0.927343,0.716003,-4.580050,-0.854225,-0.687982,-0.648986,-0.621718,-0.609839,-0.594560,-0.573805
384,3000.0,-0.895479,0.674889,-4.575528,-0.795523,-0.682855,-0.645469,-0.621447,-0.611420,-0.594922,-0.573717
512,3000.0,-0.923557,0.721335,-4.577974,-0.827268,-0.685516,-0.645257,-0.620504,-0.608127,-0.593351,-0.575889
640,3000.0,-0.918661,0.729937,-4.578732,-0.801797,-0.679082,-0.643443,-0.617904,-0.607242,-0.593055,-0.578998
768,3000.0,-0.920763,0.700350,-4.575869,-0.839764,-0.687627,-0.647426,-0.622079,-0.611702,-0.596993,-0.579669
896,3000.0,-0.924032,0.723687,-4.575528,-0.835090,-0.686143,-0.647427,-0.621746,-0.607156,-0.591599,-0.573436
1024,3000.0,-0.930260,0.727301,-4.574938,-0.840478,-0.682729,-0.645084,-0.619933,-0.607143,-0.590942,-0.577927


In [4]:
idx = df.groupby('random_state')['bayesian_opt'].idxmax()
df.loc[idx]

,random_state,nth_trial,num_exp,max_bin,max_depth,n_estimator,time,colsample_bytree,learning_rate,reg_alpha,reg_lambda,bayesian_opt,reward
2532,0,1266,2532,512,6,5000,5.844346,0.988244,0.012012,2.938685,1.820923,-0.574700,-0.574700
4363,128,681,1363,512,5,2500,2.784355,1.000000,0.041044,1.392223,1.904183,-0.573507,-0.573507
6764,256,382,764,512,6,500,0.905111,1.000000,0.106868,0.915141,2.895594,-0.573805,-0.573805
11196,384,1098,2196,512,6,500,0.913134,1.000000,0.102276,0.847358,1.658326,-0.573717,-0.573717
13920,512,960,1920,512,6,5000,6.908848,0.970691,0.018702,2.124906,2.956163,-0.575889,-0.575889
16273,640,636,1273,512,7,500,0.982707,0.985353,0.100427,0.280998,2.286881,-0.578998,-0.578998
18992,768,496,992,512,5,5000,5.210236,0.922496,0.017251,0.350587,2.199223,-0.579669,-0.579669
21473,896,236,473,512,6,1000,1.525685,1.000000,0.046204,1.803689,2.884272,-0.573436,-0.573436
25657,1024,828,1657,256,6,2500,3.341690,1.000000,0.031547,0.883833,2.896255,-0.577927,-0.577927
28902,1152,951,1902,512,6,2500,3.076141,0.903541,0.064870,0.820791,2.747112,-0.578267,-0.578267


In [11]:
import plotly.graph_objs as go

fig = go.Figure()
moving_avg_list = []
moving_lower_quantile_list = []
moving_upper_quantile_list = []
cummax_list = []
for key, gp in df.groupby('random_state'):
    moving_avg_list.append(gp.sort_values('num_exp')['bayesian_opt'].rolling(150).mean().dropna().values)
    moving_lower_quantile_list.append(gp.sort_values('num_exp')['bayesian_opt'].rolling(150).quantile(0.25).dropna().values)
    moving_upper_quantile_list.append(gp.sort_values('num_exp')['bayesian_opt'].rolling(150).quantile(0.75).dropna().values)
    cummax_list.append(gp.sort_values('num_exp')['bayesian_opt'].cummax().values)
moving_avg = pd.DataFrame(moving_avg_list).mean()
moving_lower_quantile = pd.DataFrame(moving_lower_quantile_list).mean()
moving_upper_quantile = pd.DataFrame(moving_upper_quantile_list).mean()
cummax = pd.DataFrame(cummax_list).mean()
fig.add_trace(go.Scatter(x=list(range(len(moving_avg))), y=moving_avg, mode='lines', name='mean'))
fig.add_trace(go.Scatter(x=list(range(len(moving_lower_quantile))), y=moving_lower_quantile, mode='lines', name='lower'))
fig.add_trace(go.Scatter(x=list(range(len(moving_upper_quantile))), y=moving_upper_quantile, mode='lines', name='upper'))
fig.add_trace(go.Scatter(x=list(range(len(cummax))), y=cummax, mode='lines', name='cummax'))
fig.show()

In [56]:
from ast import literal_eval as make_tuple


l = []
last_random_state = -99999
names = ['random_state', 'num_exp', 'reward']
with open('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt', 'r') as f:
    x = f.readline()
    while x:
        tmp = make_tuple(x)
        if last_random_state != tmp[0]:
            idx = 0
            last_random_state = tmp[0]
        tmp_dict = {
            'random_state': tmp[0],
            'nth_trial': tmp[1],
            'num_exp': idx,
            'time': tmp[5],
        }
        tmp_dict.update(tmp[2][1][0])
        tmp_dict.update(tmp[2][1][1])
        tmp_dict['reward'] = tmp[4]
        l.append(tmp_dict)
        idx += 1
        x = f.readline()

In [57]:
df = pd.DataFrame(l)
df.groupby('random_state')['reward'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
random_state,,,,,,,,,,,
768,3.0,-0.983268,0.284681,-1.177466,-1.146664,-1.115863,-0.88617,-0.748354,-0.702415,-0.665664,-0.656477


In [6]:
import json

data_list = []
for n in range(10):
    try:
        with open(f'/home/xyz/data/haoyan/bandits/logs/bayesian_results_{n*128}.log.json', 'r') as f:
            data = json.loads('['+f.read().replace('\n', ',')[:-1]+']')
            l = []
            for x in data:
                tmp = {'reward': x['target']}
                tmp.update(x['params'])
                l.append(tmp)  
            df = pd.DataFrame(l)
            df['random_state'] = n * 128
            df['num_exp'] = df.index
            data_list.append(df)
    except FileNotFoundError:
        pass
    
df = pd.concat(data_list)

In [7]:
df.groupby('random_state')['reward'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
random_state,,,,,,,,,,,
0,600.0,-1.040351,0.717347,-4.213611,-1.003878,-0.725043,-0.672650,-0.635667,-0.615850,-0.599865,-0.592831
128,600.0,-1.146035,0.892046,-4.579193,-1.075463,-0.735594,-0.674836,-0.639952,-0.627341,-0.598112,-0.583424
256,600.0,-1.088560,0.790492,-4.562042,-1.095192,-0.738549,-0.671077,-0.638262,-0.623381,-0.600755,-0.583299
384,600.0,-1.279395,0.998083,-4.693502,-1.316692,-0.799305,-0.686066,-0.642768,-0.627360,-0.598661,-0.587005
512,600.0,-0.902932,0.578682,-4.155275,-0.855910,-0.702594,-0.660363,-0.628429,-0.614552,-0.591785,-0.585261
640,600.0,-1.027360,0.754558,-4.379808,-0.891176,-0.727138,-0.668709,-0.635865,-0.615209,-0.592779,-0.585156
768,600.0,-0.867609,0.520516,-4.434011,-0.825560,-0.699978,-0.655070,-0.629133,-0.616338,-0.600616,-0.587890
896,600.0,-1.006535,0.679660,-4.379808,-0.909605,-0.733480,-0.677611,-0.643644,-0.624637,-0.605255,-0.587392
1024,600.0,-1.268254,1.013493,-4.651043,-1.231030,-0.759786,-0.684522,-0.645021,-0.632913,-0.610579,-0.576293


In [90]:
idx = df.groupby('random_state')['reward'].idxmax()
df.loc[idx]

,reward,colsample_bytree,learning_rate,max_bin_exp,max_depth,n_estimators,reg_alpha,reg_lambda,random_state,num_exp
29,-0.592831,0.745400,0.061695,9.711671,5.092624,2185.606601,1.913531,2.252233,0,29
29,-0.677891,0.438766,0.195117,8.491738,4.310384,4920.998317,1.923592,1.814986,128,29
29,-0.651706,0.799639,0.022158,6.178396,3.886755,4833.812456,1.066070,2.368354,256,29
29,-1.118989,0.200000,0.500000,10.000000,6.000000,4866.603848,0.000000,0.000000,384,29
29,-0.699478,0.338379,0.151563,8.424962,5.909543,2576.678703,1.037275,2.532590,512,29
...,...,...,...,...,...,...,...,...,...,...
293,-1.861662,0.347599,0.202314,8.187335,1.692861,4988.519545,2.761793,2.701350,640,293
293,-0.646443,0.519355,0.136414,8.125147,5.294693,723.773362,1.104278,1.288922,768,293
293,-0.677034,0.895768,0.440673,8.276464,5.856012,4308.161250,1.517075,1.466252,896,293
293,-0.754675,0.515689,0.457681,6.305074,5.369769,582.180505,0.479636,0.404843,1024,293


In [93]:
import plotly.graph_objs as go

fig = go.Figure()
moving_avg_list = []
moving_lower_quantile_list = []
moving_upper_quantile_list = []
cummax_list = []
for key, gp in df.groupby('random_state'):
    moving_avg_list.append(gp.sort_values('num_exp')['reward'].rolling(50).mean().dropna().values)
    moving_lower_quantile_list.append(gp.sort_values('num_exp')['reward'].rolling(50).quantile(0.25).dropna().values)
    moving_upper_quantile_list.append(gp.sort_values('num_exp')['reward'].rolling(50).quantile(0.75).dropna().values)
    cummax_list.append(gp.sort_values('num_exp')['reward'].cummax().values)
moving_avg = pd.DataFrame(moving_avg_list).mean()
moving_lower_quantile = pd.DataFrame(moving_lower_quantile_list).mean()
moving_upper_quantile = pd.DataFrame(moving_upper_quantile_list).mean()
cummax = pd.DataFrame(cummax_list).mean()
fig.add_trace(go.Scatter(x=list(range(len(moving_avg))), y=moving_avg, mode='lines', name='mean'))
fig.add_trace(go.Scatter(x=list(range(len(moving_lower_quantile))), y=moving_lower_quantile, mode='lines', name='lower'))
fig.add_trace(go.Scatter(x=list(range(len(moving_upper_quantile))), y=moving_upper_quantile, mode='lines', name='upper'))
# fig.add_trace(go.Scatter(x=list(range(len(cummax))), y=cummax, mode='lines', name='cummax'))
fig.show()

In [71]:
from ast import literal_eval as make_tuple


l = []
last_random_state = -99999
names = ['random_state', 'num_exp', 'reward']
with open('/home/xyz/data/haoyan/bandits/logs/rl_results.txt', 'r') as f:
    x = f.readline()
    while x:
        tmp = make_tuple(x)
        if last_random_state != tmp[0]:
            idx = 0
            last_random_state = tmp[0]
        tmp_dict = {
            'random_state': tmp[0],
            'nth_trial': tmp[1],
            'num_exp': idx,
            'max_bin': tmp[2][0][0],
            'n_estimator': tmp[2][0][1],
            'max_depth': tmp[2][0][2],
            'colsample_bytree': tmp[2][0][3],
            'learning_rate': tmp[2][0][4],
            'reg_alpha': tmp[2][0][5],
            'reg_lambda': tmp[2][0][6],
            'time': tmp[5],
        }
        tmp_dict['reward'] = tmp[4]
        l.append(tmp_dict)
        idx += 1
        x = f.readline()
df = pd.DataFrame(l)

In [72]:
df.groupby('random_state')['reward'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
random_state,,,,,,,,,,,
0,600.0,-1.060492,0.731452,-4.668216,-1.079079,-0.770297,-0.685048,-0.635051,-0.614823,-0.596836,-0.574712
128,600.0,-1.144649,0.839045,-4.702630,-1.171117,-0.762357,-0.682928,-0.639660,-0.622105,-0.603654,-0.586634
256,600.0,-1.081784,0.781486,-4.702711,-1.097650,-0.766776,-0.678404,-0.636587,-0.616334,-0.594112,-0.584257
384,600.0,-1.103520,0.776686,-4.702605,-1.123290,-0.777468,-0.688574,-0.645254,-0.626404,-0.598179,-0.578264
512,600.0,-1.107070,0.775945,-4.698984,-1.129063,-0.777664,-0.687999,-0.646292,-0.629994,-0.599637,-0.590415
640,600.0,-1.081880,0.754410,-4.660543,-1.097276,-0.754825,-0.681924,-0.646608,-0.627178,-0.600387,-0.574424
768,600.0,-1.052662,0.700568,-4.702085,-1.105617,-0.776438,-0.691006,-0.642065,-0.617061,-0.598488,-0.594623
896,600.0,-1.093909,0.773468,-4.701955,-1.108498,-0.773928,-0.684363,-0.637137,-0.620727,-0.596356,-0.578283
1024,600.0,-1.053412,0.699719,-4.685330,-1.089345,-0.763825,-0.684796,-0.647625,-0.627622,-0.600366,-0.592922


In [73]:
df = pd.DataFrame(l)
idx = df.groupby('random_state')['reward'].idxmax()
df.loc[idx]

,random_state,nth_trial,num_exp,max_bin,n_estimator,max_depth,colsample_bytree,learning_rate,reg_alpha,reg_lambda,time,reward
567,0,567,567,1024,2000,6,1.000000,0.104241,2.586207,2.068966,4.695886,-0.574712
728,128,128,128,512,1000,6,1.000000,0.293517,0.517241,2.379310,2.928158,-0.586634
1307,256,107,107,512,1000,6,0.911111,0.087034,0.931034,1.137931,2.824518,-0.584257
2119,384,319,319,1024,5000,6,0.911111,0.018207,2.793103,2.275862,8.989659,-0.578264
2661,512,261,261,1024,5000,5,0.911111,0.035414,0.310345,1.344828,7.244739,-0.590415
3192,640,192,192,512,2000,6,0.911111,0.052621,2.275862,2.689655,4.209376,-0.574424
3736,768,136,136,256,2000,6,0.911111,0.018207,0.413793,0.517241,3.779702,-0.594623
4355,896,155,155,1024,2000,6,1.000000,0.190276,2.689655,1.241379,4.325952,-0.578283
5150,1024,350,350,256,1000,6,0.822222,0.104241,2.896552,2.172414,2.931214,-0.592922
5433,1152,33,33,512,2000,6,0.822222,0.035414,1.448276,2.896552,4.180869,-0.583042


In [75]:
import plotly.graph_objs as go

fig = go.Figure()
moving_avg_list = []
moving_lower_quantile_list = []
moving_upper_quantile_list = []
for key, gp in df.groupby('random_state'):
    moving_avg_list.append(gp.sort_values('num_exp')['reward'].rolling(100).mean().dropna().values)
    moving_lower_quantile_list.append(gp.sort_values('num_exp')['reward'].rolling(100).quantile(0.25).dropna().values)
    moving_upper_quantile_list.append(gp.sort_values('num_exp')['reward'].rolling(100).quantile(0.75).dropna().values)
moving_avg = pd.DataFrame(moving_avg_list).mean()
moving_lower_quantile = pd.DataFrame(moving_lower_quantile_list).mean()
moving_upper_quantile = pd.DataFrame(moving_upper_quantile_list).mean()
fig.add_trace(go.Scatter(x=list(range(len(moving_avg))), y=moving_avg, mode='lines', name='mean'))
fig.add_trace(go.Scatter(x=list(range(len(moving_lower_quantile))), y=moving_lower_quantile, mode='lines', name='lower'))
fig.add_trace(go.Scatter(x=list(range(len(moving_upper_quantile))), y=moving_upper_quantile, mode='lines', name='upper'))
fig.show()

In [17]:
idx = df.groupby('random_state')['bayesian_opt'].idxmax()
df.loc[idx]

,random_state,nth_trial,num_exp,max_bin,max_depth,n_estimator,time,colsample_bytree,learning_rate,reg_alpha,reg_lambda,bayesian_opt,reward
354,0,177,354,1024,6,500,0.746513,0.937137,0.218228,1.158751,2.362864,-0.585947,-0.585947
1071,128,235,471,256,7,5000,7.238627,0.767999,0.022510,1.567936,2.766862,-0.589180,-0.589180
1342,256,71,142,512,6,5000,6.357062,0.860998,0.016522,2.188111,1.274393,-0.583187,-0.583187
2376,384,288,576,1024,7,2500,4.513225,0.773176,0.041806,1.021666,1.691569,-0.587524,-0.587524
2512,512,56,112,1024,7,500,0.917989,0.935523,0.067805,2.745755,2.580376,-0.580431,-0.580431
3524,640,262,524,512,5,1000,1.165357,0.888625,0.101009,1.660038,1.803643,-0.586439,-0.586439
3736,768,68,136,256,7,5000,7.453038,0.912430,0.046040,1.101532,2.904342,-0.589684,-0.589684
4743,896,271,543,1024,7,1000,1.927257,0.938103,0.034008,1.275749,2.214812,-0.592711,-0.592711
5378,1024,289,578,512,6,500,0.665495,0.927675,0.152545,1.297683,2.415112,-0.577484,-0.577484
5636,1152,118,236,512,7,1000,1.700618,0.722974,0.093315,0.545417,1.494211,-0.591243,-0.591243


In [18]:
df.loc[idx]['num_exp'].mean()

367.2

In [22]:
df.loc[idx]['num_exp'].mean()

224.8